In [5]:
from flask import Flask
from flask import request,Response
import os
import sqlite3
import pandas as pd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import io
import numpy as np

In [20]:
def getAll():
    conn = sqlite3.connect('stock.db')
    strSQL="""
    SELECT s.Symbol, d.Date_Time, p.Open, p.High, p.Low, p.Close, p.Volume, p.Dividends, p.Stock_Splits, p.Return, p.Acc_Return 
    FROM Price p
    JOIN Symbols s ON p.Symbol_ID = s.Symbol_ID
    JOIN Dates d ON p.Date_ID = d.Date_ID
    """
    df = pd.read_sql(strSQL, conn)
    conn.close()
    return df

def showSymbols():
    conn = sqlite3.connect('stock.db')
    strSQL="SELECT DISTINCT Symbol FROM Symbols"
    df = pd.read_sql(strSQL, conn)
    conn.close()
    return df

def getSymbol(sym):
    conn = sqlite3.connect('stock.db')
    strSQL="""
    SELECT s.Symbol, d.Date_Time, p.Open, p.High, p.Low, p.Close, p.Volume, p.Dividends, p.Stock_Splits, p.Return, p.Acc_Return 
    FROM Price p
    JOIN Symbols s ON p.Symbol_ID = s.Symbol_ID
    JOIN Dates d ON p.Date_ID = d.Date_ID
    WHERE s.Symbol=:sym
    """
    df = pd.read_sql_query(strSQL, conn, params={"sym": sym})
    conn.close()
    return df


def get_stock_data(date_time):
    conn = sqlite3.connect('stock.db')
    strSQL = """
    SELECT s.Symbol, p.Open, p.High, p.Low, p.Close, p.Volume, p.Dividends, p.Stock_Splits, p.Return, p.Acc_Return 
    FROM Price p
    JOIN Symbols s ON p.Symbol_ID = s.Symbol_ID
    JOIN Dates d ON p.Date_ID = d.Date_ID
    WHERE d.Date_Time LIKE :date_time || '%'
    """
    df = pd.read_sql_query(strSQL, conn, params={"date_time": date_time})
    conn.close()

    return df

In [19]:
app = Flask(__name__)

@app.route("/",methods=['GET'])
def index():
    df = getAll()
    return Response(df.to_json(orient="records"), mimetype='application/json')

@app.route("/AllSymbols",methods=['GET'])
def getSYMS():
    df = showSymbols()
    return Response(df.to_json(orient="records"), mimetype='application/json')

@app.route("/Symbol/<sym>", methods=['GET'])
def getSYM(sym):
    df = getSymbol(sym)
    return Response(df.to_json(orient="records"), mimetype='application/json')

@app.route("/StockData/<date_time>", methods=['GET'])
def getStockData(date_time):
    df = get_stock_data(date_time)
    return Response(df.to_json(orient="records"), mimetype='application/json')

if __name__=="__main__":
    app.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 5000)))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.3:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2024 22:17:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2024 22:17:16] "GET /StockData/2024-03-20 HTTP/1.1" 200 -
